# Tratamento e limpez de dados
Nesta etapa serão feitas diversas tarefas iniicias para realização da análise dos dados do projeto

## Primeiro dia
- unificar em um único Dataframe todos os dados pertinentes para a análise
- Limpar dados diplicados e nulos

In [56]:
# imports

import pandas as pd
from glob import glob

In [57]:
# Leitura dos arquivos CSV na pasta dados_emprestimos e unificação em um único DataFrame

file_list = glob('../datasets/dados_emprestimos/*.csv')
df_loans = pd.concat((pd.read_csv(file) for file in file_list), ignore_index=True)

df_loans.head()

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
0,9405484,2011010806,2017/01/13 07:22:40.736000000,2017/01/02 08:01:14.113000000,2017/01/30 07:05:05.000000000,2.014036e+09,ALUNO DE GRADUAÇÃO
1,9405488,L078593,2017/01/31 07:31:36.960000000,2017/01/02 08:29:46.657000000,2017/02/21 13:16:48.000000000,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO
2,9405492,2010040100,2017/01/31 07:31:36.902000000,2017/01/02 08:29:46.722000000,2017/02/21 13:16:33.000000000,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO
3,9405496,2013105705,2017/01/31 07:31:36.843000000,2017/01/02 08:29:46.794000000,2017/02/21 13:15:46.000000000,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO
4,9405500,L057436,2017/01/31 07:30:36.293000000,2017/01/02 08:38:40.009000000,2017/02/21 13:16:29.000000000,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO


In [58]:
# Verificando se existem valores duplicados no DataFrame

duplicated_rows = df_loans.duplicated()
num_duplicated = duplicated_rows.sum()
print(f'Número de linhas duplicadas: {num_duplicated}')

Número de linhas duplicadas: 37


In [59]:
# excluindo linhas duplicadas e removendo valores NaN na coluna 'id_emprestimo'

df_loans.drop_duplicates(inplace=True)
df_loans['id_emprestimo'].dropna(inplace=True)

In [60]:
# Verificando novamente se existem valores duplicados após a limpeza

duplicated_rows =df_loans.duplicated().sum()
print(f'Número de linhas duplicadas após limpeza: {duplicated_rows}')


Número de linhas duplicadas após limpeza: 0


In [61]:
# verificando se existem valores NaN na coluna 'id_emprestimo'

nan_count = df_loans['id_emprestimo'].isna().sum()
print(f'Número de valores NaN na coluna id_emprestimo: {nan_count}')

Número de valores NaN na coluna id_emprestimo: 0


In [62]:
# transformando dados NaN em zeros no DataFrame

df_loans.fillna(0, inplace=True)

In [63]:
# transformando as colunas de datas em formato datetime

df_loans['data_renovacao'] = pd.to_datetime(df_loans['data_renovacao'], errors='coerce')
df_loans['data_devolucao'] = pd.to_datetime(df_loans['data_devolucao'], errors='coerce')
df_loans['data_emprestimo'] = pd.to_datetime(df_loans['data_emprestimo'], errors='coerce')


In [64]:
# Lendo os dados do acervo de livros

df_books = pd.read_parquet('../datasets/dados_acervo/dados_exemplares.parquet')
df_books.head()

,id_exemplar,codigo_barras,colecao,biblioteca,status_material,localizacao,registro_sistema
index,,,,,,,
0,5,L000003,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,694,1
1,4,L000002,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,688,1
2,3,L000001,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,638,1
3,7,L000114,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,616,5
5,10,L000041,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,657,15


In [65]:
# Verificando se existem valores duplicados no DataFrame de livros

duplicated_rows_books = df_books.duplicated().sum()
print(f'Número de linhas duplicadas no DataFrame de livros: {duplicated_rows_books}')

Número de linhas duplicadas no DataFrame de livros: 0


In [66]:
# verificando se exestem valores NaN no DataFrame de livros

nan_count_books = df_books.isna().sum()
print(f'Número de valores NaN no DataFrame de livros:\n{nan_count_books}')

Número de valores NaN no DataFrame de livros:
id_exemplar         0
codigo_barras       0
colecao             0
biblioteca          0
status_material     0
localizacao         0
registro_sistema    0
dtype: int64


In [67]:
# unindo os DataFrames de empréstimos e livros com base na coluna 'codigo_barras'

df_combined = pd.merge(df_loans, df_books, left_on='codigo_barras', right_on='codigo_barras', how='inner')
df_combined.head()

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,registro_sistema
0,9405484,2011010806,2017-01-13 07:22:40.736,2017-01-02 08:01:14.113,2017-01-30 07:05:05,2.014036e+09,ALUNO DE GRADUAÇÃO,915480,Acervo Circulante,Biblioteca Setorial Profª. Maria José Mamede G...,REGULAR,689,165557
1,9405492,2010040100,2017-01-31 07:31:36.902,2017-01-02 08:29:46.722,2017-02-21 13:16:33,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO,1051275,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,651,171079
2,9405496,2013105705,2017-01-31 07:31:36.843,2017-01-02 08:29:46.794,2017-02-21 13:15:46,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO,1097159,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,380,187099
3,9405504,H004983,2017-01-31 07:31:36.786,2017-01-02 08:46:42.991,2017-02-22 16:10:28,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO,132715,Acervo Circulante,Biblioteca Setorial do Centro de Ciências Huma...,REGULAR,635,44994
4,9405508,H011333,2017-01-31 07:31:36.658,2017-01-02 08:46:43.400,2017-02-22 16:10:40,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO,180475,Acervo Circulante,Biblioteca Setorial do Centro de Ciências Huma...,REGULAR,632,69163


In [68]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2072522 entries, 0 to 2072521
Data columns (total 13 columns):
 #   Column                Dtype         
---  ------                -----         
 0   id_emprestimo         int64         
 1   codigo_barras         object        
 2   data_renovacao        datetime64[ns]
 3   data_emprestimo       datetime64[ns]
 4   data_devolucao        datetime64[ns]
 5   matricula_ou_siape    float64       
 6   tipo_vinculo_usuario  object        
 7   id_exemplar           int64         
 8   colecao               object        
 9   biblioteca            object        
 10  status_material       object        
 11  localizacao           int64         
 12  registro_sistema      int64         
dtypes: datetime64[ns](3), float64(1), int64(4), object(5)
memory usage: 205.6+ MB


## Segundo dia

* criar uma nova coluna com os valores da localização, para refletir a respectiva classe geral na CDU
* excluir coluna "registro_sistema"
* Transformar em formato String a coluna da matricula (“matricula_ou_siape”)

In [69]:
# criando uma coluna para localização do livro de acordo com a classe geral CDU

cdu_mapping = {
    "000 a 099": "Generalidades. Ciência e conhecimento",
    "100 a 199": "Filosofia e psicologia",
    "200 a 299": "Religião",
    "300 a 399": "Ciências sociais",
    "400 a 499": "Classe vaga. Provisoriamente não ocupada",
    "500 a 599": "Matemática e ciências naturais",
    "600 a 699": "Ciências aplicadas",
    "700 a 799": "Belas artes",
    "800 a 899": "Linguagem. Linguística. Literatura",
    "900 a 999": "Geografia. Biografia. História",
}


location_books = df_combined['localizacao']
df_combined['classificacao_cdu'] = ''

df_combined.loc[df_combined['localizacao'] <= 99, 'classificacao_cdu'] = cdu_mapping["000 a 099"]
df_combined.loc[(df_combined['localizacao'] >= 100) & (df_combined['localizacao'] <= 199), 'classificacao_cdu'] = cdu_mapping["100 a 199"]
df_combined.loc[(df_combined['localizacao'] >= 200) & (df_combined['localizacao'] <= 299), 'classificacao_cdu'] = cdu_mapping["200 a 299"]
df_combined.loc[(df_combined['localizacao'] >= 300) & (df_combined['localizacao'] <= 399), 'classificacao_cdu'] = cdu_mapping["300 a 399"]
df_combined.loc[(df_combined['localizacao'] >= 400) & (df_combined['localizacao'] <= 499), 'classificacao_cdu'] = cdu_mapping["400 a 499"]
df_combined.loc[(df_combined['localizacao'] >= 500) & (df_combined['localizacao'] <= 599), 'classificacao_cdu'] = cdu_mapping["500 a 599"]
df_combined.loc[(df_combined['localizacao'] >= 600) & (df_combined['localizacao'] <= 699), 'classificacao_cdu'] = cdu_mapping["600 a 699"]
df_combined.loc[(df_combined['localizacao'] >= 700) & (df_combined['localizacao'] <= 799), 'classificacao_cdu'] = cdu_mapping["700 a 799"]
df_combined.loc[(df_combined['localizacao'] >= 800) & (df_combined['localizacao'] <= 899), 'classificacao_cdu'] = cdu_mapping["800 a 899"]
df_combined.loc[(df_combined['localizacao'] >= 900) & (df_combined['localizacao'] <= 888), 'classificacao_cdu'] = cdu_mapping["900 a 999"]

df_combined.head(10)

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,registro_sistema,classificacao_cdu
0,9405484,2011010806,2017-01-13 07:22:40.736,2017-01-02 08:01:14.113,2017-01-30 07:05:05,2.014036e+09,ALUNO DE GRADUAÇÃO,915480,Acervo Circulante,Biblioteca Setorial Profª. Maria José Mamede G...,REGULAR,689,165557,Ciências aplicadas
1,9405492,2010040100,2017-01-31 07:31:36.902,2017-01-02 08:29:46.722,2017-02-21 13:16:33,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO,1051275,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,651,171079,Ciências aplicadas
2,9405496,2013105705,2017-01-31 07:31:36.843,2017-01-02 08:29:46.794,2017-02-21 13:15:46,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO,1097159,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,380,187099,Ciências sociais
3,9405504,H004983,2017-01-31 07:31:36.786,2017-01-02 08:46:42.991,2017-02-22 16:10:28,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO,132715,Acervo Circulante,Biblioteca Setorial do Centro de Ciências Huma...,REGULAR,635,44994,Ciências aplicadas
4,9405508,H011333,2017-01-31 07:31:36.658,2017-01-02 08:46:43.400,2017-02-22 16:10:40,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO,180475,Acervo Circulante,Biblioteca Setorial do Centro de Ciências Huma...,REGULAR,632,69163,Ciências aplicadas
5,9405512,L193660,2017-01-30 10:14:56.636,2017-01-02 08:55:12.453,2017-03-02 14:02:56,2.280658e+06,DOCENTE,161744,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,639,60092,Ciências aplicadas
6,9405516,2011078083,2017-01-30 10:14:56.547,2017-01-02 08:55:12.516,2017-03-02 14:03:01,2.280658e+06,DOCENTE,961156,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,636,166330,Ciências aplicadas
7,9405520,2014049788,2017-01-30 10:14:56.473,2017-01-02 08:55:12.575,2017-03-02 14:02:52,2.280658e+06,DOCENTE,1146450,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,339,174097,Ciências sociais
8,9405524,2012051946,2017-01-16 08:47:52.594,2017-01-02 09:29:09.656,2017-01-31 14:54:31,1.953639e+06,SERVIDOR TÉCNICO-ADMINISTRATIVO,1020065,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,330,187679,Ciências sociais
9,9405528,2010069476,NaT,2017-01-02 09:41:45.823,2017-01-23 16:30:59,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO,820497,Acervo Circulante,Biblioteca Setorial Prof. Ronaldo Xavier de Ar...,REGULAR,623,68991,Ciências aplicadas


In [70]:
# excluindo a coluna 'registro_sistema' que não é necessária para a análise

df_combined.drop(columns=['registro_sistema'], inplace=True)
df_combined.head()

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,classificacao_cdu
0,9405484,2011010806,2017-01-13 07:22:40.736,2017-01-02 08:01:14.113,2017-01-30 07:05:05,2.014036e+09,ALUNO DE GRADUAÇÃO,915480,Acervo Circulante,Biblioteca Setorial Profª. Maria José Mamede G...,REGULAR,689,Ciências aplicadas
1,9405492,2010040100,2017-01-31 07:31:36.902,2017-01-02 08:29:46.722,2017-02-21 13:16:33,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO,1051275,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,651,Ciências aplicadas
2,9405496,2013105705,2017-01-31 07:31:36.843,2017-01-02 08:29:46.794,2017-02-21 13:15:46,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO,1097159,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,380,Ciências sociais
3,9405504,H004983,2017-01-31 07:31:36.786,2017-01-02 08:46:42.991,2017-02-22 16:10:28,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO,132715,Acervo Circulante,Biblioteca Setorial do Centro de Ciências Huma...,REGULAR,635,Ciências aplicadas
4,9405508,H011333,2017-01-31 07:31:36.658,2017-01-02 08:46:43.400,2017-02-22 16:10:40,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO,180475,Acervo Circulante,Biblioteca Setorial do Centro de Ciências Huma...,REGULAR,632,Ciências aplicadas


In [71]:
# transformando o tipo de dado da coluna 'matricula_ou_siape' para string
df_combined['matricula_ou_siape'] = df_combined['matricula_ou_siape'].astype('int64').astype('str')
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2072522 entries, 0 to 2072521
Data columns (total 13 columns):
 #   Column                Dtype         
---  ------                -----         
 0   id_emprestimo         int64         
 1   codigo_barras         object        
 2   data_renovacao        datetime64[ns]
 3   data_emprestimo       datetime64[ns]
 4   data_devolucao        datetime64[ns]
 5   matricula_ou_siape    object        
 6   tipo_vinculo_usuario  object        
 7   id_exemplar           int64         
 8   colecao               object        
 9   biblioteca            object        
 10  status_material       object        
 11  localizacao           int64         
 12  classificacao_cdu     object        
dtypes: datetime64[ns](3), int64(3), object(7)
memory usage: 205.6+ MB


In [72]:
df_combined.head()

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,classificacao_cdu
0,9405484,2011010806,2017-01-13 07:22:40.736,2017-01-02 08:01:14.113,2017-01-30 07:05:05,2014035816,ALUNO DE GRADUAÇÃO,915480,Acervo Circulante,Biblioteca Setorial Profª. Maria José Mamede G...,REGULAR,689,Ciências aplicadas
1,9405492,2010040100,2017-01-31 07:31:36.902,2017-01-02 08:29:46.722,2017-02-21 13:16:33,20161011168,ALUNO DE PÓS-GRADUAÇÃO,1051275,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,651,Ciências aplicadas
2,9405496,2013105705,2017-01-31 07:31:36.843,2017-01-02 08:29:46.794,2017-02-21 13:15:46,20161011168,ALUNO DE PÓS-GRADUAÇÃO,1097159,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,380,Ciências sociais
3,9405504,H004983,2017-01-31 07:31:36.786,2017-01-02 08:46:42.991,2017-02-22 16:10:28,20161011168,ALUNO DE PÓS-GRADUAÇÃO,132715,Acervo Circulante,Biblioteca Setorial do Centro de Ciências Huma...,REGULAR,635,Ciências aplicadas
4,9405508,H011333,2017-01-31 07:31:36.658,2017-01-02 08:46:43.400,2017-02-22 16:10:40,20161011168,ALUNO DE PÓS-GRADUAÇÃO,180475,Acervo Circulante,Biblioteca Setorial do Centro de Ciências Huma...,REGULAR,632,Ciências aplicadas


In [73]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2072522 entries, 0 to 2072521
Data columns (total 13 columns):
 #   Column                Dtype         
---  ------                -----         
 0   id_emprestimo         int64         
 1   codigo_barras         object        
 2   data_renovacao        datetime64[ns]
 3   data_emprestimo       datetime64[ns]
 4   data_devolucao        datetime64[ns]
 5   matricula_ou_siape    object        
 6   tipo_vinculo_usuario  object        
 7   id_exemplar           int64         
 8   colecao               object        
 9   biblioteca            object        
 10  status_material       object        
 11  localizacao           int64         
 12  classificacao_cdu     object        
dtypes: datetime64[ns](3), int64(3), object(7)
memory usage: 205.6+ MB


In [74]:
# Salvando o DataFrame final em um arquivo Parquet

df_combined.to_parquet('../datasets/dados_tratados/dados_emprestimos_tratados.parquet', engine='pyarrow', compression='snappy', index=False)